# IEOR 174 Assignment 1: Deliverable 2

Joy Zhang

In [1]:
#import necessary packages
import sklearn as sk # For models
import numpy as np # For data manipulation
import pandas as pd # For data manipulation
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For plotting
import math
import random
from collections import Counter
import scipy.stats as stats
np.random.seed(0)

> Simulate n = 50 independent days of operation from 11AM and 1PM and generate the sequence of customer waiting times. For this task, the service time distribution F is given by an exponential distribution with expectation as 35 seconds. Recall that the condition λ = 2 persons per minute in the problem statement implies that the exponentially distributed inter-arrival times of customers have expectation also as 30
seconds.

### Defining Variables and Simulation

In [2]:
# vars for reference

lambda_rate = 2  # Rate of arrivals per minute
opening = 0  # open at 11am
closing = 120  # close at 1pm
exp_expectation= 35 # E[F] = 35 customers per minute
expectation_interarrival = 30 
service_mean = 35 / 60  # Average service time in minutes (35 secs)
arrival_mean = 30 / 60  # Average inter-arrival time in minutes (30 secs)

In [3]:
# Simulate customer waiting times for each individual day
def simulate_day(arrival_time_mean, service_time_mean, start_time, end_time):
    arrival_times = []
    service_times = []
    wait_times = []
    departure_times = []
    
    current_time = start_time  # Initialize current time at the start time (in seconds)
    departure_time = 0  # Time when the last customer finishes service
    
    while current_time < end_time:
        # Generate the next inter-arrival time and service time
        inter_arrival_time = np.random.exponential(arrival_time_mean)
        service_time = np.random.exponential(service_time_mean)
        
        # Update the current time for the next customer's arrival
        current_time += inter_arrival_time
        
        # Stop the simulation if the arrival time exceeds the end time
        if current_time >= end_time:
            break
        
        # Calculate the waiting time
        if current_time < departure_time:
            wait_time = departure_time - current_time  # Customer waits
        else:
            wait_time = 0  # No wait, customer is served immediately
        
        # Update departure time after the current customer is served
        departure_time = current_time + wait_time + service_time
        
        # Store the times
        arrival_times.append(current_time)
        service_times.append(service_time)
        wait_times.append(wait_time)
        departure_times.append(departure_time)
    
    
    return arrival_times, service_times, wait_times, departure_times  # In minutes


## Part i

### Running Simulation

In [4]:
def simulate_50_days():
    all_arrival_times = []
    all_service_times = []
    all_waiting_times = []
    all_departure_times = []

    for _ in range(50):
        arrival_times, service_times, waiting_times, departure_times = simulate_day(arrival_mean, service_mean, 0, 120)
        
        # Collect all arrival, service, waiting, and departure times across all days
        all_arrival_times.extend(arrival_times)
        all_service_times.extend(service_times)
        all_waiting_times.extend(waiting_times)
        all_departure_times.extend(departure_times)

    return all_arrival_times, all_service_times, all_waiting_times, all_departure_times


arrival_times1, service_times1, waiting_times1, departure_times1 = simulate_50_days()


i.) Suppose that you are interested in knowing that on a typical Wednesday, what is the expectation of the averaged waiting time for customers who arrive at the food truck between 11:15 AM to 11:30 AM. Compute an estimate of this quantity using simulation.

In [5]:
# Time Window
arrival_start_time1 = 15  # 11:15 AM in minutes
arrival_end_time1 = 30    # 11:30 AM in minutes

In [6]:
def estimated_wait_times(arrival_times, waiting_times, arrival_start_time, arrival_end_time):
    relevant_waiting_times = []

    for i, arrival_time in enumerate(arrival_times):
        if arrival_start_time <= arrival_time <= arrival_end_time:
            relevant_waiting_times.append(waiting_times[i])

    # Calculate the average of relevant waiting times
    average_wait_time = np.mean(relevant_waiting_times) if relevant_waiting_times else 0

    return relevant_waiting_times, average_wait_time


In [7]:
relevant_waiting_times1, average_wait_time1 = estimated_wait_times(arrival_times1, waiting_times1, arrival_start_time1, arrival_end_time1)

In [8]:
# Estimate average waiting time for first time frame
print(f"{average_wait_time1:.2f} minutes")


6.09 minutes


### Calculating Confidence Interval:

In [9]:
def average_wait_time_with_CI(rel_waiting_times):

    if len(rel_waiting_times) > 0:
        mean_wait_time = np.mean(rel_waiting_times)
        std_dev = np.std(rel_waiting_times, ddof=1)  # Sample standard deviation
        n = len(rel_waiting_times)  # Sample size
        
        # Calculate the critical value
        critical_value = stats.t.ppf(0.975, n - 1)  # 95% confidence level
        
        # Calculate margin of error
        margin_of_error = critical_value * (std_dev / np.sqrt(n))
        
        # Construct the confidence interval
        lower_bound = mean_wait_time - margin_of_error
        upper_bound = mean_wait_time + margin_of_error


    return (lower_bound, upper_bound)


In [10]:
ci = average_wait_time_with_CI(relevant_waiting_times1)
# CI for first time frame
ci

(5.863098593002203, 6.322095848017348)

## Part ii

Suppose that you are interested in knowing that on a typical Wednesday, what is the expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM. Compute an estimate of this quantity using simulation.

In [11]:
# Time Window
arrival_start_time2 = 105  # 12:45 PM in minutes
arrival_end_time2 = 120    # 1:00 PM in minutes

In [12]:
# Average waiting time for customers arriving between 12:45 PM to 1:00 PM
relevant_waiting_times2, average_wait_time2 = estimated_wait_times(arrival_times1, waiting_times1, arrival_start_time2, arrival_end_time2)
print(f"{average_wait_time2:.2f} minutes")

22.42 minutes


## Part iii

Compare the two quantities you computed in the previous two questions. Describe the intuition that you may get from this comparison.

The arrival rate does not change and is fixed at 2 persons per minute with an inter-arrival time expectation of 30 seconds and a service time distribution given by an exponential distribution with an expectation of 35 seconds. Both periods (11:15 to 11:30 and 12:45 to 1:00) have the same arrival rate, but their respective positions within the 2-hour window of the shop being in business (11:00 AM to 1:00 PM) can affect the system dynamics.

The average waiting time between 11:15 AM and 11:45 AM is expected to be lower because the system starts fresh, with fewer customers building up.
The average waiting time between 12:45 PM and 1:15 PM is likely higher due to the system's accumulated load, even though the arrival rate remains constant.


This comparison shows how, even with a constant arrival rate, the system’s state over time can affect waiting times. Early periods benefit from an "empty system," while later periods experience the cumulative effect of earlier arrivals.

## Part iv

Compute the percentage of customers who arrive at the food truck between 11:15 AM to 11:30 AM and wait for more than 3 minutes. Compute the percentage of customers who arrive at the food truck between 12:45 PM to 1:00 PM and wait for more than 3 minutes.

In [13]:
def compute_waiting_time_percentages(waiting_times, threshold=3):
    total_customers = len(waiting_times)
    long_waits_count = sum(1 for wait_time in waiting_times if wait_time > threshold)

    # Calculate the percentage of customers with long waits
    if total_customers > 0:
        percentage = (long_waits_count / total_customers) * 100
    else:
        percentage = 0  # no customers

    return percentage


In [14]:
percentages1 = compute_waiting_time_percentages(relevant_waiting_times1, threshold=3)
percentages2 = compute_waiting_time_percentages(relevant_waiting_times2, threshold=3)

print(f"Percentage of customers waiting more than 3 minutes from 11:15 to 11:30 AM : {percentages1:.2f}%")
print(f"Percentage of customers waiting more than 3 minutes from 12:45 to 1:00 PM : {percentages2:.2f}%")

Percentage of customers waiting more than 3 minutes from 11:15 to 11:30 AM : 74.30%
Percentage of customers waiting more than 3 minutes from 12:45 to 1:00 PM : 96.73%


## Part v

Suppose that the customers will immediately abandon the system and leave for other dining options, conditional on that they see more than 5 people in the system (including the one being served). 

Now, what is the percentage of customers who abandon the system (=food truck) upon arrival between 12:45 PM and 1:00 PM? 

What is the expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM and did not abandon the system?

In [15]:
# vars from earlier
arrival_start_time1 = 15  
arrival_end_time1 = 30 

arrival_start_time2 = 105
arrival_end_time2 = 120

In [16]:
def simulate_abandonment(arrival_times, service_times, waiting_times, departure_times, start_time, end_time, max_customers_in_system=5):
    n_customers = len(arrival_times)
    
    abandonment_count = 0
    queue = []  # Tracks customers who are being processed
    non_abandoned_wait_times = []  # Tracks wait times of non-abandoned customers

    for i in range(n_customers):
        # Skip customers arriving before the start time
        if arrival_times[i] < start_time:
            continue
        
        # Stop the loop if the customer arrives after the end time
        if arrival_times[i] > end_time:
            break
        
        # Process and remove customers from the queue who have already departed
        while queue and queue[0] <= arrival_times[i]:
            queue.pop(0)
        
        # If the system is full (queue has max capacity), customer abandons
        if len(queue) >= max_customers_in_system:
            abandonment_count += 1
            continue
        
        # Add the non-abandoned customer to the queue
        queue.append(departure_times[i])  # Departure time of the current customer
        
        # Store the waiting time for this non-abandoned customer
        non_abandoned_wait_times.append(waiting_times[i])
    
    # Calculate the percentage of customers who abandoned
    abandonment_percentage = (abandonment_count / n_customers) * 100
    
    # Calculate average waiting time for non-abandoned customers
    avg_wait_time_non_abandoned = np.mean(non_abandoned_wait_times) if non_abandoned_wait_times else 0
    
    return abandonment_percentage, avg_wait_time_non_abandoned


In [17]:
abandonment_percentage, mean_non_abandoned_wait_time = simulate_abandonment(arrival_times1, service_times1, waiting_times1, departure_times1, arrival_start_time2, arrival_end_time2, max_customers_in_system=5)

print(f"Percentage of customers who abandon the system (12:45 PM to 1:00 PM): {abandonment_percentage:.2f}%")
print(f"Average wait time for non-abandoned customers (12:45 PM to 1:00 PM): {mean_non_abandoned_wait_time:.2f} minutes")


Percentage of customers who abandon the system (12:45 PM to 1:00 PM): 12.13%
Average wait time for non-abandoned customers (12:45 PM to 1:00 PM): 16.91 minutes


## Part vi

Re-do the previous five parts with F being an exponential distribution with expectation as 30 seconds and everything else equal.

In [18]:
# vars 

lambda_rate = 2  # Rate of arrivals per minute
opening = 0  # opens at 11am
closing = 120  # closes at 1pm
exp_expectation= 30 # E[F] = 30 customers per minute
expectation_interarrival = 30 
service_mean = 30 / 60  # Average service time in minutes (30 secs)
arrival_mean = 30 / 60  # Average inter-arrival time in minutes (30 secs)

i) Expectation of the averaged waiting time for customers who arrive at the food truck between 11:15 AM to 11:30 AM

In [19]:
arrival_times5, service_times5, waiting_times5, departure_times5 = simulate_50_days()

print("Arrival Times:", arrival_times5[:10])
print("Service Times:", service_times5[:10])
print("Waiting Times:", waiting_times5[:10])
print("Departure Times:", departure_times5[:10])

Arrival Times: [2.001319764414644, 3.263580934230097, 4.093540056622911, 5.1645661753343015, 5.257246354775418, 5.3444702767599965, 5.438711082113072, 6.846207446898903, 8.718334221257937, 8.999660053884224]
Service Times: [1.3340385920118518, 1.1429506826774043, 0.6370999428439746, 0.017334367489752856, 0.8947406787125681, 0.46029355653778475, 0.3899464518875156, 0.47085171577842977, 0.26576711130288, 0.33726052372209553]
Waiting Times: [0, 0.07177742219639871, 0.38476898248098923, 0, 0, 0.8075167567279902, 1.173569507912699, 0.15601959501438412, 0, 0]
Departure Times: [3.3353583564264957, 4.4783090391039, 5.1154089819478745, 5.181900542824055, 6.151987033487987, 6.6122805900257715, 7.0022270419132875, 7.4730787576917175, 8.984101332560817, 9.33692057760632]


In [20]:
arrival_start_time1 = 15  # 11:15 AM in minutes
arrival_end_time1 = 30    # 11:30 AM in minutes

relevant_waiting_times51, average_wait_time = estimated_wait_times(arrival_times5, waiting_times5, arrival_start_time1, arrival_end_time1)

print(f"{average_wait_time:.2f} minutes")

4.28 minutes


ii) Expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM

In [21]:
arrival_start_time2 = 105  # 12:45 PM in minutes
arrival_end_time2 = 120  # 1:00 PM in minutes

relevant_waiting_times52, average_wait_time = estimated_wait_times(arrival_times5, waiting_times5, arrival_start_time2, arrival_end_time2)

print(f"{average_wait_time:.2f} minutes")

9.31 minutes


iii) Compare the two quantities you computed in the previous two questions. Describe intuition that you may get from this comparison.

Similarly, like the previous response to the differences between the two averages, we observe the same pattern when the service exponential distribution has an expectation of 30 seconds and with everything else equal. The pattern of the time range 12:45 to 1:00 PM having a higher average waiting time compared to 11:15 to 11:30 AM is apparent once again for the same reason.

The difference between the two averages with the changed expectation is actually on average relatively smaller than our previous simulation. With the service exponential distribution having a lower expectation of 30 seconds instead of 35 seconds, it can be inferred that it takes 5 seconds less on average to service customers now than before.

With a 30-second service time, the overall system moves more efficiently, especially during the busier period (12:45 PM to 1:15 PM), which will bring the average waiting times in the two periods closer together.
Therefore, the difference between the average waiting times in the two periods is likely to be smaller with a 30-second service time than with a 35-second service time.

iv) Compute the percentage of customers who arrive at the food truck between 11:15 AM to 11:30 AM and wait for more than 3 minutes. Compute the percentage of customers who arrive at the food truck between 12:45 PM to 1:00 PM and wait for more than 3 minutes.

In [22]:
percentages51 = compute_waiting_time_percentages(relevant_waiting_times51, threshold=3)
percentages52 = compute_waiting_time_percentages(relevant_waiting_times52, threshold=3)

print(f"Percentage of customers waiting more than 3 minutes from 11:15 to 11:30 AM : {percentages51:.2f}%")
print(f"Percentage of customers waiting more than 3 minutes from 12:45 to 1:00 PM : {percentages52:.2f}%")


Percentage of customers waiting more than 3 minutes from 11:15 to 11:30 AM : 51.69%
Percentage of customers waiting more than 3 minutes from 12:45 to 1:00 PM : 78.10%


v) Suppose that the customers will immediately abandon the system and leave for other dining options, conditional on that they see more than 5 people in the system (including the one being served). Now, what is the percentage of customers who abandon the system (=food truck) upon arrival between 12:45 PM and 1:00 PM? What is the expectation of the averaged waiting time for customers who arrive at the food truck between 12:45 PM to 1:00 PM and did not abandon the system?

In [23]:
abandonment_percentage, mean_non_abandoned_wait_time = simulate_abandonment(arrival_times5, service_times5, waiting_times5, departure_times5, arrival_start_time2, arrival_end_time2, max_customers_in_system=5)

print(f"Percentage of customers who abandon the system (12:45 PM to 1:00 PM): {abandonment_percentage:.2f}%")
print(f"Average wait time for non-abandoned customers (12:45 PM to 1:00 PM): {mean_non_abandoned_wait_time:.2f} minutes")


Percentage of customers who abandon the system (12:45 PM to 1:00 PM): 12.88%
Average wait time for non-abandoned customers (12:45 PM to 1:00 PM): 7.80 minutes


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=253bc843-1c66-4ac2-ba68-7bc57bcd9b5b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>